### Введение
С развитием LLM-моделей спрашивать теоретическую часть или короткие алгоритмы и принципы стало бесполезно, поэтому задание будет одно, но комплексное и приближенное к реальности.

Вы можете использовать любые LLM-модели и Copilot для написания кода, так как в реальной работе сотрудники отдела также имеют доступ ко всем современным инструментам (Sonnet 3.5, GPT-4o, GitHub Copilot и т. п.).

В задании вы будете работать с тестовыми данными по банкоматам и API ЦБ РФ для получения ключевой ставки.

---

### Специфика данных fees
`fees` — тарифы за обслуживание банкомата инкассаторами. Так как банкоматы могут находиться в разных регионах, тарифы у них могут отличаться.

- **CashDeliveryFixedFee** — фиксированная стоимость доставки наличных (не зависит от суммы).
- **CashDeliveryPercentageFee** — процент от суммы доставленных наличных. Он добавляется к фиксированной стоимости. Если в таблице указано `0.0001`, это значит 0.01% от суммы.
- **CashDeliveryMinFee** — минимальная сумма, которую с нас возьмут по `CashDeliveryPercentageFee`. Например, если мы попросим инкассаторов в ATM_4 довезти всего 10 000 рублей, то с нас возьмут 5250 рублей (фиксированная стоимость), а так как `10 000 * 0.04% < 450 рублей`, то дополнительно возьмут ещё 450 рублей. Итог: 5250 + 450 рублей.

**CashCollection** — это пример сложного тарифа, который учитывает специфику работы с банкоматом. В банкомат деньги не докладывают, а меняют сразу кассету. Есть провайдеры, которые тарифицируют не только доставку, но и пересчёт денег в извлечённой кассете. Например, в ATM_4 у нас именно такой тариф.  
Пример: у нас в банкомате осталось 500 тыс. рублей, а мы хотим, чтобы у него был баланс 2 млн. Тогда мы заказываем довезти кассету на 2 млн. и платим по тарифу за доставку. При доставке старая кассета извлекается, и мы также по тарифу платим за пересчёт 500 тыс. рублей (0.45%, но не менее 1140 рублей).

- **CashCollectionFixedFee** — фиксированная стоимость за извлечение старой кассеты (в данных примерах нулевая).
- **CashCollectionPercentageFee** — процент от суммы извлечённой кассеты.
- **CashCollectionMinFee** — минимальная сумма, которую с нас возьмут по `CashCollectionPercentageFee`.

---

### Специфика данных transactions
Таблица содержит данные по снятиям, пополнениям (инкассациям) и балансу банкомата на конец дня.  
Считаем, что банкоматы в начале года пустые и не работали, поэтому баланс на конец дня равен 0, пока не случится первая инкассация.

- **bal_end_of_day** — баланс на конец дня.
- **cash_in** — пополнение в результате инкассации.
- **cash_out** — снятие наличных клиентами.

---

In [1]:
import pandas as pd

# Забираем данные по тарфиам на обслуживание банкоматов
fees = pd.read_parquet('https://storage.yandexcloud.net/norvpublic/fees.parquet')
# статистика операция по дням.
transactions = pd.read_parquet('https://storage.yandexcloud.net/norvpublic/transactions.parquet')

### Часть 1 — упущенный процентный доход

Специалисту по ML важно уметь получать данные с различных API и читать документацию. Для расчёта упущенного процентного дохода нужно обратиться к API ЦБ РФ и получить динамику ключевой ставки за 2024 год.

https://www.cbr.ru/DailyInfoWebServ/DailyInfo.asmx?op=KeyRate

Для запроса к API не нужен токен и регистрация. ЦБ РФ для части данных предпочитает использовать SOAP.

Учитывая, что хранить наличные деньги в банкомате — не самое удачное инвестиционное решение, посчитайте упущенный процентный доход для каждого банкомата. Рассчитываем, что банк мог бы вложить эти деньги и получить доход, равный ключевой ставке ЦБ РФ, актуальной на день баланса банкомата. Добавьте к таблице `transactions` столбец с упущенной процентной выгодой.


In [ ]:
import requests
from xml.etree import ElementTree as ET
from datetime import datetime
import re

# SOAP XML запрос
soap_request = f"""
<soap12:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:soap12="http://www.w3.org/2003/05/soap-envelope">
  <soap12:Body>
    <KeyRate xmlns="http://web.cbr.ru/">
      <fromDate>2024-01-01T00:00:00</fromDate>
      <ToDate>2024-12-31T23:59:59</ToDate>
    </KeyRate>
  </soap12:Body>
</soap12:Envelope>
"""

# Заголовки для запроса
headers = {
    'Content-Type': 'application/soap+xml; charset=utf-8',
    'Content-Length': str(len(soap_request))
}

# Отправка SOAP запроса
response = requests.post('http://www.cbr.ru/DailyInfoWebServ/DailyInfo.asmx', headers=headers, data=soap_request)

xml_data = response.content.decode('utf-8')
pattern = re.compile(r'<DT>(.*?)</DT>\s*<Rate>(.*?)</Rate>')
matches = pattern.findall(xml_data)


date_arr = []
rate_arr = []

for date, rate in matches:
    date_arr.append(date)
    rate_arr.append(rate)
    
date_arr_ymd = []

for i in range(0, len(date_arr)):
    original_string = date_arr[i]
    parsed_date = datetime.strptime(original_string, '%Y-%m-%dT%H:%M:%S%z').date()

# Преобразуем в строку в формате 'YYYY-MM-DD'
    formatted_date = parsed_date.strftime('%Y-%m-%d')
    date_arr_ymd.append(formatted_date)

# Создание словаря
rate_dict = dict(zip(date_arr_ymd, rate_arr))

# Преобразование формата даты
transactions['date_only'] = pd.to_datetime(transactions['date']).dt.date.astype(str)

# Функция для вычисления упущенной процентной прибыли
def calculate_missed_income(data_dict, df):
    df['lost profits'] = 0  

    for index, row in df.iterrows():
        date = row['date_only']
        balance = row['bal_end_of_day']

        # Если дата есть в словаре, рассчитаем упущенную прибыль
        if date in data_dict:
            key_rate = data_dict[date]
            df.at[index, 'lost profits'] = (balance * float(key_rate) / 100 / 365)
    return df

transactions = calculate_missed_income(rate_dict, transactions)
transactions = transactions.drop('date_only', axis=1)
print(transactions['lost profits'])

### Часть 2 - расходы на инкассацию

Теперь к данным по банкоматам, кроме упущенного процентного дохода, необходимо добавить столбец расходов на инкассацию. Для простоты считаем, что мы не платим за аренду помещения и страхование, поэтому нам достаточно данных по тарифам на инкассацию (`fees`).

In [ ]:
import pandas as pd
import numpy as np

fees = fees.fillna(0)
# Создаем новый столбец с предыдущим балансом на конец дня
transactions['previous_bal_end_of_day'] = transactions.groupby('ATM_ID')['bal_end_of_day'].shift(1)

# Объединяем данные о тарифах с данными о транзакциях
merged_df = pd.merge(transactions, fees, on='ATM_ID', how='left')

# 1. Расчет расходов на доставку наличных
merged_df['cash_delivery_cost'] = merged_df['cash_in'] * merged_df['CashDeliveryPercentageFee'] + merged_df['CashDeliveryFixedFee']

merged_df['cash_delivery_cost'] = np.where(
    ~merged_df['CashDeliveryMinFee'].isna(),
    np.maximum(merged_df['cash_delivery_cost'], merged_df['CashDeliveryMinFee']),
    merged_df['cash_delivery_cost']
)

# 2. Расчет расходов на извлечение старой кассеты
merged_df['cash_collection_cost'] = merged_df['previous_bal_end_of_day'] * merged_df['CashCollectionPercentageFee']

# Если предыдущий баланс равен 0, то пересчет кассеты стоит 0
merged_df['cash_collection_cost'] = np.where(
    merged_df['previous_bal_end_of_day'] == 0, 
    0, 
    merged_df['cash_collection_cost']
)


merged_df['cash_collection_cost'] = np.where(
    ~merged_df['CashCollectionMinFee'].isna(),
    np.maximum(merged_df['cash_collection_cost'], merged_df['CashCollectionMinFee']),
    merged_df['cash_collection_cost']
)

# 3. Итоговая стоимость инкассации
merged_df['incassation_cost'] = merged_df['cash_delivery_cost'] + merged_df['cash_collection_cost']

merged_df['incassation_cost'] = np.where(merged_df['cash_in'] == 0, 0, merged_df['incassation_cost'])

transactions['incassation_cost'] = merged_df['incassation_cost'].fillna(0)
transactions = transactions.drop(['previous_bal_end_of_day'], axis = 1)

# Вывод результата
print(transactions['incassation_cost'])

### Часть 3 - анализ данных

Посмотрите на данные о снятиях, инкассациях и балансе. В данных подобраны банкоматы, отличающиеся по характеру использования и частоте инкассаций.  
(Эту операцию вы делаете для себя, чтобы лучше понять специфику.)

### Часть 4 - меняем бизнес с помощью ML

Заключительная и самая интересная часть. К данному этапу у нас уже рассчитан упущенный процентный доход и расходы на инкассацию. Подумайте, как использовать ваши знания, чтобы оптимизировать процесс инкассации и уменьшить издержки.

Для данной задачи считаем, что нас устроит Service Level на уровне 90%. Это значит, что если в 9 из 10 случаев наши клиенты получают нужные им суммы, нас это устраивает.

Напоминаем, что технически в банкомат нельзя довнести сумму, и кассета меняется полностью. Соответственно, с нас берут оплату за полную кассету (если вдруг решите подойти к задаче через классическую формулу EOQ).

Вы можете подойти к задаче абсолютно любым способом и использовать все возможности ООП, Python и любых библиотек. Всё как в реальной работе, где вас никто не ограничивает.

Эффективность своего решения вы можете показать с помощью ретротестирования. Считаем, что каждая кассета может вмещать абсолютно любые суммы. Чтобы добавить реализма, вы можете считать, что деньги нужно заказывать за 3 дня до их доставки.


### Заключение

Решение вы можете предоставить любым способом: репозиторий GitHub, Google Colab, конвертированный IPython Notebook в PDF и т. п.

По срокам вас не ограничивают, но учитывайте, что в это же время задачу могут решать и другие кандидаты. Таким образом, вы участвуете в конкурсе с ними.